In [20]:
## https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

import requests
import os
import json
import pandas as pd
import numpy as np
import csv
import datetime
import dateutil.parser
import unicodedata
import time
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [22]:
def auth():
    #return os.getenv('BEARER_TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def append_tweet_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet and Conversation ID
        tweet_id = tweet['id']
        conversation_id = tweet['conversation_id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [tweet_id, conversation_id, author_id, created_at, geo, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter)

In [27]:
# Get selected news accounts
news_accounts = pd.read_csv("./../data/external/20211117_selected_news_accounts.csv", header=None)[0].values

# Create tweet file and write header
csvFile = open("./../data/processed/news_tweets.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['tweet_id', 'conversation_id', 'author_id', 'created_at', 'geo', 'lang', 'like_count', 'quote_count', 'reply_count', 'retweet_count', 'source', 'text'])
csvFile.close()

# Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
start_date = '2020-01-01T00:00:00.000Z'
end_date = '2021-10-31T23:59:59.000Z'
max_results = 500

# Loop Inputs
total_tweets = 0
count = 0 # Counting tweets per time period
max_count = 10000000 # Max tweets per time period
flag = True
next_token = None
n_requests = 0

# Check if flag is true
start_time = time.time()

In [44]:
news_accounts[160:]

array(['gmanews', 'rapplerdotcom', 'ANI', 'ndtv', 'ndtvfeed',
       'timesofindia', 'TimesNow', 'htTweets', 'republic', '9NewsAUS',
       '9NewsSyd', '9NewsMelb', 'SkyNewsAust', 'abcnews', 'newscomauHQ',
       'News24', 'eNCA', 'TimesLIVE', 'SABCNews', 'rtenews',
       'Independent_ie', 'IrishTimes', 'IrishTimesWorld', 'thejournal_ie',
       'MobilePunch', 'vanguardngrnews', 'PulseNigeria247',
       'citizentvkenya', 'ntvkenya', 'NationAfrica', 'ntvuganda', 'nbstv',
       'DailyMonitor', 'malaysiakini', 'staronline', 'NewshubNZ',
       'nzherald'], dtype=object)

In [ ]:
start_time=time.time()

In [47]:
start_time=time.time()
while flag:
    # Check if max_count reached
    if count >= max_count:
        break
    print("-------------------")
    print("Request #: ", n_requests+1)
    print("Requesting Time: ", int(time.time() - start_time))
    print("Token: ", next_token)
    url = create_url(keyword, start_date,end_date, max_results)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
        # Save the token to use for next call
        next_token = json_response['meta']['next_token']
        print("Next Token: ", next_token)
        if result_count is not None and result_count > 0 and next_token is not None:
            append_tweet_to_csv(json_response, "./../data/processed/news_tweets.csv")
            count += result_count
            total_account_tweets += result_count
            total_tweets += result_count
            n_requests += 1
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(1)                
    # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-------------------")
            append_tweet_to_csv(json_response, "./../data/processed/news_tweets.csv")
            count += result_count
            total_account_tweets += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(1)

        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
    time.sleep(1)
    # If reachs requests cap, stop it
    if n_requests%295==0 and time.time()-start_time<900:
        time.sleep(900 - (time.time()-start_time))
    elif n_requests%295==0  and time.time()-start_time>=900:
        start_time=time.time()
print("Total number of account results: ", total_account_tweets)

for account in news_accounts[160:]:
    
    keyword = f"context:123.1220701888179359745 from:{account} lang:en -is:retweet -is:reply"
    print("--------------------------------------")
    print("--------------------------------------")
    print("News Account: ", account)
    total_account_tweets = 0
    flag = True
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Request #: ", n_requests+1)
        print("Requesting Time: ", int(time.time() - start_time))
        print("Token: ", next_token)
        url = create_url(keyword, start_date,end_date, max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                append_tweet_to_csv(json_response, "./../data/processed/news_tweets.csv")
                count += result_count
                total_account_tweets += result_count
                total_tweets += result_count
                n_requests += 1
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(1)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                append_tweet_to_csv(json_response, "./../data/processed/news_tweets.csv")
                count += result_count
                total_account_tweets += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(1)

            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(1)
        # If reachs requests cap, stop it
        if n_requests%295==0 and time.time()-start_time<900:
            time.sleep(900 - (time.time()-start_time))
        elif n_requests%295==0  and time.time()-start_time>=900:
            start_time=time.time()
    print("Total number of account results: ", total_account_tweets)
print("Total number of results: ", total_tweets)

-------------------
Request #:  2546
Requesting Time:  0
Token:  b26v89c19zqg8o3fo71h6dzpuqvcb1dv0aq27vu0r6g71
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2jtkwlcz5r0g8axfott1k19x9
# of Tweets added from this response:  500
Total # of Tweets added:  1306787
-------------------
-------------------
Request #:  2547
Requesting Time:  4
Token:  b26v89c19zqg8o3fo71f2jtkwlcz5r0g8axfott1k19x9
Endpoint Response Code: 200
-------------------
# of Tweets added from this response:  247
Total # of Tweets added:  1307034
-------------------
Total number of account results:  25747
--------------------------------------
--------------------------------------
News Account:  gmanews
-------------------
Request #:  2547
Requesting Time:  7
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpds7pd4hm57hks1ob543139a6ngxp
# of Tweets added from this response:  500
Total # of Tweets added:  1307534
-------------------
-------------------
Request #:  2548
Requesting Time: 